<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 8.0 NER Model Deployment with Riva
## (part of Lab 2)

In this notebook, you'll begin with a custom `.riva` restaurant NER model.  You'll then convert it to an optimized model with the Riva ServiceMaker framework, which aggregates the necessary artifacts for deployment to a target environment. Finally, you'll connect the deployed model to a demo web app and see it in action!

**[8.1 Deployment Recap](#8.1-Deployment-Recap)<br>**
**[8.2 Riva ServiceMaker](#8.2-Riva-ServiceMaker)<br>**
**[8.3 Riva Server](#8.3-Riva-Server)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[8.3.1 Exercise: Riva Configuration Custom NER](#8.3.1-Exercise:-Riva-Configuration-Custom-NER)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[8.3.2 Start Riva Services](#8.3.2-Start-Riva-Services)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[8.3.3 Riva Available Services Check](#8.3.3-Riva-Available-Services-Check)<br>
**[8.4 Riva NLP Service Request](#8.4-Riva-NLP-Service-Request)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[8.4.1 Python Client Demo](#8.4.1-Python-Client-Demo)<br>
**[8.5 Restaurant NER with the Riva Contact App](#8.5-Restaurant-NER-with-the-Riva-Contact-App)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[8.5.1 Stop, Reconfigure, and Restart Riva](#8.5.1-Stop,-Reconfigure,-and-Restart-Riva)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[8.5.1.1 Exercise: Enable ASR in config.sh](#8.5.1.1-Exercise:-Enable-ASR-in-config.sh)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[8.5.1.2 Restart Riva Services](#8.5.1.2-Restart-Riva-Services)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[8.5.2 Update and Start Riva Contact App](#8.5.2-Update-and-Start-Riva-Contact-App)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[8.5.2.1 Exercise: Update `env.txt`](#8.5.2.1-Exercise:-Update-env.txt)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[8.5.2.2 Start the Contact Web Server](#8.5.2.2-Start-the-Contact-Web-Server)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[8.5.3 Stop Riva Services](#8.5.3-Stop-Riva-Services)<br>

### Notebook Dependencies
To run this app on your system, you will need:
1. **Microphone**<br>
For best ASR results, a headset is recommended.  
1. **Chrome browser**<br>
In order to use the app over HTTP in our class setup, you will need to override the browser block to your camera and microphone.  Instructions are included later in the notebook.
1. **NGC Credentials**<br>Be sure you have added your NGC credential as described in the [NGC Setup notebook](003_Intro_NGC_Setup.ipynb)
2. **exported-model-NER.riva**<br>
Execute the next cell to load a backup copy of the exported Riva NER model into the correct location if it was not  exported in the previous notebook with `tao token_classification export`.

In [1]:
%%bash
mkdir -p /dli/task/tao/results/export
cp /dli/task/tao/backup_riva/exported-model-NER.riva /dli/task/tao/results/export

In [2]:
# Check running docker containers. This should be empty.
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [ ]:
# If not empty,
# Clear Docker containers to start fresh...
!docker kill $(docker ps -q)
# Check for clean environment - this should be empty
!docker ps

---
# 8.1 Deployment Recap
Deployment with Riva ServiceMaker `riva-build` and `riva-deploy` commands work the same way for our new NER model as for the ASR model exported in an earlier notebook.  To recap, the components of deployment are:
1. **Riva ServiceMaker**
   - Set up path variables for a clean workflow
   - Run the ServiceMaker container with `riva-build` for the token_classification to combine `.riva` files and create a RMIR file
   - Run the ServiceMaker container with `riva-deploy` to deploy the RMIR file to a specified target model repository
1. **Riva Server**
    - Configure the `config.sh` file for your application
    - Initialize the server with `javis_init.sh`
    - Start the Riva server with `riva_start.sh`

---
# 8.2 Riva ServiceMaker
Set up the path variables.

In [3]:
# Set the WORKSPACE path to "/path/to/your/workspace"
WORKSPACE = "/dli/task"

# ServiceMaker Docker
RIVA_SM_CONTAINER = "nvcr.io/nvidia/riva/riva-speech:1.4.0-beta-servicemaker"

# Directory where the .riva model is stored $MODEL_LOC/*.riva
MODEL_LOC = WORKSPACE + '/tao/results/export'

# Directory where the .rmir model is stored $RMIR_LOC/*.rmir
RIVA_MODEL_LOC = WORKSPACE + '/riva/riva_quickstart/models_repo_NER'
RMIR_LOC = RIVA_MODEL_LOC + "/rmir"

# Name of the .erjvs file
EXPORT_MODEL_NAME = "exported-model-NER.riva"
RMIR_MODEL_NAME = "token-classification.rmir"

# Key that model is encrypted with, while exporting with TAO
KEY='tlt_encode'

In [4]:
# Syntax: riva-build <task-name> output-dir-for-rmir/model.rmir:key dir-for-riva/model.riva:key
!docker run --rm --gpus 1 \
    -v $MODEL_LOC:/tao \
    -v $RMIR_LOC:/riva \
    $RIVA_SM_CONTAINER -- \
    riva-build token_classification /riva/$RMIR_MODEL_NAME:$KEY /tao/$EXPORT_MODEL_NAME:$KEY


=== Riva Speech Skills ===

NVIDIA Release devel (build 22382700)

Copyright (c) 2018-2021, NVIDIA CORPORATION.  All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION.  All rights reserved.
NVIDIA modifications are covered by the license terms that apply to the underlying
project or file.

NOTE: Legacy NVIDIA Driver detected.  Compatibility mode ENABLED.

NOTE: The SHMEM allocation limit is set to the default of 64MB.  This may be
   insufficient for the inference server.  NVIDIA recommends the use of the following flags:
   nvidia-docker run --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 ...

2022-06-22 12:58:24,438 [ERROR] Condition for key 'runtime' (PyTorch  <built-in function eq> ONNX) is not fulfilled
2022-06-22 12:58:24,509 [INFO] Packing binaries for label_tokens
2022-06-22 12:58:25,696 [ERROR] Condition for key 'runtime' (PyTorch  <built-in function eq> ONNX) is not fulfilled
2022-06-22 12:58:25,757 [INFO] Trying to extract from model exp

In [5]:
# check the token-classification RMIR file
!ls $RMIR_LOC

token-classification.rmir


In [6]:
# Syntax: riva-deploy -f dir-for-rmir/model.rmir:key output-dir-for-repository
!docker run --rm --gpus 1 \
     -v $RIVA_MODEL_LOC:/data \
     $RIVA_SM_CONTAINER -- \
     riva-deploy -f  /data/rmir/$RMIR_MODEL_NAME:$KEY /data/models/


=== Riva Speech Skills ===

NVIDIA Release devel (build 22382700)

Copyright (c) 2018-2021, NVIDIA CORPORATION.  All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION.  All rights reserved.
NVIDIA modifications are covered by the license terms that apply to the underlying
project or file.

NOTE: Legacy NVIDIA Driver detected.  Compatibility mode ENABLED.

NOTE: The SHMEM allocation limit is set to the default of 64MB.  This may be
   insufficient for the inference server.  NVIDIA recommends the use of the following flags:
   nvidia-docker run --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 ...

2022-06-22 13:14:46,587 [INFO] Writing Riva model repository to '/data/models/'...
2022-06-22 13:14:46,587 [INFO] The riva model repo target directory is /data/models/
2022-06-22 13:14:47,752 [INFO] Extract_binaries for tokenizer -> /data/models/riva_tokenizer/1
2022-06-22 13:14:47,754 [INFO] Extract_binaries for language_model -> /data/models/riva-trt-riva_

In [7]:
# Check optimized models 
!ls $RIVA_MODEL_LOC/models

riva-trt-riva_ner-nn-bert-base-uncased	riva_ner	       riva_tokenizer
riva_detokenize				riva_ner_label_tokens


[`riva-ner/config.pbtxt`](riva/riva_quickstart/models_repo_NER/models/riva-ner/config.pbtxt) describes the model input/output format and the ensemble scheduling for the NER task, which includes the following models:

- Tokenizer `riva_tokenizer`
- NER `riva-trt-riva_ner-nn-bert-base-uncased`
- Mapper for labels to predictions `riva_ner_label_tokens`
- Detokenizer `riva_detokenize`

<img src="images/ner/riva_ner.png"><br>

---
# 8.3 Riva Server

Once the model repository is generated, we are ready to start the Riva server. As before with ASR, we rely on the scripts from the [Riva Quick Start](https://ngc.nvidia.com/catalog/resources/nvidia:riva:riva_quickstart) scripts from NGC (preloaded for this course).  

Set the path to the directory here:

In [8]:
# Set the Riva Quick Start directory
RIVA_QS = WORKSPACE + "/riva/riva_quickstart"

## 8.3.1 Exercise: Riva Configuration Custom NER

Once again, we must modify [config.sh](riva/riva_quickstart/config.sh) for this particular deployment.  We need to:
* Enable NLP services for token classification.
* Provide the encryption key (should already be correct)
* Provide the path to the model repository where we generated the RMIR model in the previous step, which is now located at `/dli/task/riva/riva_quickstart/models_repo_NER`

Edit [config.sh](riva/riva_quickstart/config.sh) and make changes where necessary. 

Check your work against the [solution](solutions/ex8.3.1.sh) before moving on to the next section.  You can verify it with `diff` in the next cell. You should get no "difference" (an empty output) if your config file matches the solution.

In [10]:
# TODO modify config.sh so that this cell verifies changes are correct
# There should be no output if the files match
!diff $RIVA_QS/config.sh solutions/ex8.3.1.sh

## 8.3.2 Start Riva Services
Initialize and start the Riva server with NLP. 

In [11]:
# Initialize Riva
!cd $RIVA_QS && bash riva_init.sh config.sh

Logging into NGC docker registry if necessary...
Pulling required docker images if necessary...
Note: This may take some time, depending on the speed of your Internet connection.
> Pulling Riva Speech Server images.
  > Image nvcr.io/nvidia/riva/riva-speech:1.4.0-beta-server exists. Skipping.
  > Image nvcr.io/nvidia/riva/riva-speech-client:1.4.0-beta exists. Skipping.
  > Image nvcr.io/nvidia/riva/riva-speech:1.4.0-beta-servicemaker exists. Skipping.

Note: this may take some time, depending on the speed of your Internet connection.
To skip this process and use existing RMIRs set the location and corresponding flag in config.sh.

=== Riva Speech Skills ===

NVIDIA Release devel (build 22382700)

Copyright (c) 2018-2021, NVIDIA CORPORATION.  All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION.  All rights reserved.
NVIDIA modifications are covered by the license terms that apply to the underlying
project or file.

NOTE: Legacy NVIDIA Driver detected.  Compa

In [12]:
# Run Riva Start. This will deploy the model(s).
!cd $RIVA_QS && bash riva_start.sh config.sh

Starting Riva Speech Services. This may take several minutes depending on the number of models deployed.
Waiting for Riva server to load all models...retrying in 10 seconds
Waiting for Riva server to load all models...retrying in 10 seconds
Riva server is ready...


The Riva NLP service should be running when you get `Riva server is ready...` (about 30 seconds)

## 8.3.3 Riva Available Services Check

Check available NLP models. You should see this:

<img src="images/ner/riva_ner_logs.png">

In [ ]:
!docker logs riva-speech

---
## 8.4 Riva NLP Service Request

After the Riva server is up and running with your models, you can query the server. 
To send gRPC requests, Riva Python API bindings for the client must be installed. This is available as a pip wheel with the quick start directory.  For this class, the API is already installed.

## 8.4.1 Python Client Demo
The following cell creates a Python file that queries the Riva server (using gRPC) to yield a result.

In [13]:
%%writefile $RIVA_QS/ner_client.py

import grpc
import os
import argparse
import riva_api.riva_nlp_pb2 as rnlp
import riva_api.riva_nlp_pb2_grpc as rnlp_srv

# use the NER network to return top-1 classes for entities
def postprocess_labels_server(tokens_response):
    results = []
    for i in range(0, len(tokens_response.results)):
        slots = []
        slot_scores = []
        tokens = []
        for j in range(0, len(tokens_response.results[i].results)):
          entity = tokens_response.results[i].results[j]
          tokens.append(entity.token)
          slots.append(entity.label[0].class_name)
          slot_scores.append(entity.label[0].score)
        results.append((slots, tokens, slot_scores))

    return results

def run_ner(grpc_server, query):
    channel = grpc.insecure_channel(grpc_server)
    riva_nlp = rnlp_srv.RivaLanguageUnderstandingStub(channel)
    req = rnlp.AnalyzeEntitiesRequest()
    req.query = query
    resp = riva_nlp.AnalyzeEntities(req)
    print("Query:", query)
    print(postprocess_labels_server(resp))

def get_args():
    parser = argparse.ArgumentParser(description="Client app to test named entity recognition on Riva")
    parser.add_argument("--server", default="localhost:50051", type=str, help="URI to GRPC server endpoint")
    parser.add_argument("--model", default="riva_ner", type=str, help="Model on Riva Server to execute")
    parser.add_argument("--query", default="NVIDIA is located at Santa Clara", type=str, help="Input Query")
    return parser.parse_args()


def run_ner_client():
    args = get_args()
    run_ner(args.server, query=args.query)

if __name__ == '__main__':
    run_ner_client()

Writing /dli/task/riva/riva_quickstart/ner_client.py


Query the NLP service for NER

In [14]:
!python3 $RIVA_QS/ner_client.py --query " I would like to order a pizza at 8pm"

Query:  I would like to order a pizza at 8pm
[(['Dish', 'Hours'], ['pizza', '8pm'], [0.9604489803314209, 0.8666989803314209])]


---
# 8.5 Restaurant NER with the Riva Contact App
The Riva Contact app requires both ASR and NER to run, and we need to let the app know what entities we are tracking with our custom model.  Therefore, we need to:
1. Riva Server:
    - Stop Riva Services
    - Copy the ASR models we need into the same directory as our custom NER model
    - Reconfigure `config.sh` to include ASR
    - Start Riva Services
2. Riva Contact App
    - Modify [env.txt](contact-app/env.txt) for the Restaurant entities
    - Re-install the contact app
    - Start the app server
    - Open the app!

## 8.5.1 Stop, Reconfigure, and Restart Riva

In [15]:
# Run Riva Stop. 
!bash $RIVA_QS/riva_stop.sh

Shutting down docker containers...


In [16]:
# Copy the ASR models to our model repo
!cp -rf $WORKSPACE/riva/models/quartznet-asr-trt-ensemble-vad-streaming $RIVA_MODEL_LOC/models/
!cp -rf $WORKSPACE/riva/models/quartznet-asr-trt-ensemble-vad-streaming-feature-extractor-streaming $RIVA_MODEL_LOC/models/
!cp -rf $WORKSPACE/riva/models/riva-trt-quartznet $RIVA_MODEL_LOC/models/
!cp -rf $WORKSPACE/riva/models/quartznet-asr-trt-ensemble-vad-streaming-voice-activity-detector-ctc-streaming $RIVA_MODEL_LOC/models/
!cp -rf $WORKSPACE/riva/models/quartznet-asr-trt-ensemble-vad-streaming-ctc-decoder-cpu-streaming $RIVA_MODEL_LOC/models/

In [17]:
# Check optimized models in repo
!ls $RIVA_MODEL_LOC/models

quartznet-asr-trt-ensemble-vad-streaming
quartznet-asr-trt-ensemble-vad-streaming-ctc-decoder-cpu-streaming
quartznet-asr-trt-ensemble-vad-streaming-feature-extractor-streaming
quartznet-asr-trt-ensemble-vad-streaming-voice-activity-detector-ctc-streaming
riva-trt-quartznet
riva-trt-riva_ner-nn-bert-base-uncased
riva_detokenize
riva_ner
riva_ner_label_tokens
riva_tokenizer


### 8.5.1.1 Exercise: Enable ASR in `config.sh`
You are good at this by now!  Modify [config.sh](riva/riva_quickstart/config.sh) to enable ASR services in addition to NLP services.  The model repo is the same as before since we copied the ASR models to that directory.  Check your work against the [solution](solutions/ex8.5.1.1.sh) before restarting Riva.

In [19]:
# TODO modify config.sh so that this cell verifies changes are correct
# There should be no output if the files match
!diff $RIVA_QS/config.sh solutions/ex8.5.1.1.sh

### 8.5.1.2 Restart Riva Services

In [20]:
# Initialize Riva
!cd $RIVA_QS && bash riva_init.sh config.sh

Logging into NGC docker registry if necessary...
Pulling required docker images if necessary...
Note: This may take some time, depending on the speed of your Internet connection.
> Pulling Riva Speech Server images.
  > Image nvcr.io/nvidia/riva/riva-speech:1.4.0-beta-server exists. Skipping.
  > Image nvcr.io/nvidia/riva/riva-speech-client:1.4.0-beta exists. Skipping.
  > Image nvcr.io/nvidia/riva/riva-speech:1.4.0-beta-servicemaker exists. Skipping.

Note: this may take some time, depending on the speed of your Internet connection.
To skip this process and use existing RMIRs set the location and corresponding flag in config.sh.

=== Riva Speech Skills ===

NVIDIA Release devel (build 22382700)

Copyright (c) 2018-2021, NVIDIA CORPORATION.  All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION.  All rights reserved.
NVIDIA modifications are covered by the license terms that apply to the underlying
project or file.

NOTE: Legacy NVIDIA Driver detected.  Compa

In [21]:
# Run Riva Start. This will deploy the model(s).
!cd $RIVA_QS && bash riva_start.sh config.sh

Starting Riva Speech Services. This may take several minutes depending on the number of models deployed.
Waiting for Riva server to load all models...retrying in 10 seconds
Waiting for Riva server to load all models...retrying in 10 seconds
Riva server is ready...


In [22]:
# Check available services
!docker logs riva-speech


=== Riva Speech Skills ===

NVIDIA Release 21.07 (build 25292380)

Copyright (c) 2018-2021, NVIDIA CORPORATION.  All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION.  All rights reserved.
NVIDIA modifications are covered by the license terms that apply to the underlying
project or file.

NOTE: Legacy NVIDIA Driver detected.  Compatibility mode ENABLED.

NOTE: The SHMEM allocation limit is set to the default of 64MB.  This may be
   insufficient for the inference server.  NVIDIA recommends the use of the following flags:
   nvidia-docker run --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 ...

  > Riva waiting for Triton server to load all models...retrying in 1 second
I0622 13:26:36.869200 73 metrics.cc:228] Collecting metrics for GPU 0: Tesla T4
I0622 13:26:36.872480 73 onnxruntime.cc:1722] TRITONBACKEND_Initialize: onnxruntime
I0622 13:26:36.872507 73 onnxruntime.cc:1732] Triton TRITONBACKEND API version: 1.0
I0622 13:26:36.872515 73 onnxruntim

## 8.5.2 Update and Start Riva Contact App
Take a look at the configuration file for the contact app.  Note the NER entities listed by default:

In [23]:
!cat contact-app/env.txt

# Configuration environment vars for Riva Contact

# Replace the IP and port with your hosted RIVA endpoint
RIVA_API_URL="0.0.0.0:50051"
# NER model to use. This one is the default from the Riva Quick Start setup
RIVA_NER_MODEL="riva_ner"
# NER entities to use from the above model (can be a subset of what is offered)
RIVA_NER_ENTITIES="per,loc,org,time,misc"
#RIVA_NER_ENTITIES="amenity,dish,hours,location,price,rating,restaurant_name"

# The port your Node.js app will be hosted at
PORT="8009"
# Port for the peer-js server, to be used for negotiating the peer-to-peer chat connection
PEERJS_PORT="9000"


### 8.5.2.1 Exercise: Update `env.txt`

The first step is to update the entities that will be visually identified in the application.
Modify the [contact-app/env.txt](contact-app/env.txt) file section on entities by changing the comments to the correct restaurant context model.  Check your work against the [solution](solutions/ex8.5.2.1.txt) before starting the web app server.

In [25]:
# TODO modify env.txt so that this cell verifies changes are correct
# There should be no output if the files match
!diff contact-app/env.txt solutions/ex8.5.2.1.txt

### 8.5.2.2 Start the Contact Web Server
To start the web service, open a JupyterLab terminal.  You can do this by first opening the JupyterLab Launcher (small '+' sign at the top of the file browser) and clicking the "Terminal" icon.  Next, enter the following in the terminal to start the app server:  

```sh
cd /dli/task/contact-app
npm install
npm run start
```

In [26]:
# Run this cell, then click the link to open a terminal
# Enter the commands provided above in the terminal window to start the web server
from IPython.display import HTML
HTML('<a href="", data-commandlinker-command="terminal:create-new">Open Terminal</a>')

In the terminal window, you should see that the server has started running on port 8009:

<img src="images/asr/webserver_running.png">

After you have started the server, execute the following cell to create a link to open the app! 

In [27]:
%%js
const href = window.location.hostname + '/app/';
let a = document.createElement('a');
let link = document.createTextNode('Open Riva Contact!');
a.appendChild(link);
a.href = "http://" + href;
a.style.color = "navy"
a.target = "_blank"
element.append(a);

<IPython.core.display.Javascript object>

### Browser Restrictions Reminder

To use the web app, access to your microphone is required (camera is optional).<br>
Several browsers restrict camera/microphone access to applications served from a secure origin (HTTPS or local IP).
For your own development purposes, you can set up a self-signed certificate or proxy.  

Some browsers provide a way to treat specific URLs as secure:
- Chrome browser: <br>
Configure the "treat insecure origin as secure" flag by adding the application URL on the following page: <br>
   ***chrome://flags/#unsafely-treat-insecure-origin-as-secure***<br>
(Copy and paste this "chrome://" link to a tab on your browser to open the page) <br>
You'll see the flag with a text window at the top of the page.  Add your own course URL to the box.  Here is an example (your URL is different).<br>
More discussion can be found in [this blog](https://medium.com/@Carmichaelize/enabling-the-microphone-camera-in-chrome-for-local-unsecure-origins-9c90c3149339).

<img src="images/asr/chrome_override_example.png">

- Safari browser: <br>
Enable in the menu: Develop > WebRTC > Allow Media Capture on Insecure Sites


<img src="images/ner/riva_contact_ner2.PNG" width=800px>

Go to the application URL. Once the page is loaded, you're welcome to start the Riva transcription.
After you open the app, you may see a "Lost connection to server" alert.  Just click "Ok".  Other than that, your initial view should look like the following:
In the box titled "Riva transcription," hit the Start button, then start speaking. You'll see in-progress transcripts in the text field at the bottom. As those transcripts are finalized, they'll appear, with NER annotations, in the transcription box.

## 8.5.3 Stop Riva Services 

In [28]:
# Shut down Riva 
!bash $RIVA_QS/riva_stop.sh
# Shut down web app
!pkill -9 node

Shutting down docker containers...


---
<h2 style="color:green;">Congratulations!</h2>

In this notebook, you have:
- Exported the NER model to a RMIR format
- Configured and exposed NLP Riva services 
- Requested NLP service using a Python client API
- Demonstrated a custom NER model with ASR streaming yourself!

This concludes the NER portion of the course.<br>
Next, you'll work with deployment of Riva at scale using Kubernetes, starting with [Enabling GPU within Kubernetes](009_K8s_Enable.ipynb).

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>